<H1 align="center">Multi Layer Perceptron for Disease Spread Prediction</H1>

<strong>This notebook trains a multi-lyaer perceptron-based classification model on the training dataset and predicts on the inference dataset.</strong>

## Import Libraries

### Python Libraries

### External Libraries

In [ ]:
import torch
from torch.nn import ReLU, BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader

from utils_classification.models.TrainerMultiLayerPerceptron import TrainerMultiLayerPerceptron

### Custom Libraries

In [ ]:
from utils_classification.models import ModelMultiLayerPerceptron
from utils_classification.data import DatasetMultiLayerPerceptron
from utils_torch.data import stratified_random_split
from utils_data import *
from utils_plot import plot_confusion_matrix, plot_roc

## Set Parameters

### Select Features

In [ ]:
features = ['Normalized_Age',
            'Normalized_Behaviour',
            'Normalized_Constitution',
            'Normalized_Degree',
            'Normalized_Distance_to_Index_Patient',
            'Normalized_Sum_Neighbor_Age',
            'Normalized_Sum_Neighbor_Behaviour',
            'Normalized_Sum_Neighbor_Constitution',
            'Normalized_Sum_Neighbor_Degree',
            'Normalized_Sum_Population_Age',
            'Normalized_Sum_Population_Behaviour',
            'Normalized_Sum_Population_Constitution',
            'Normalized_Sum_Population_Distance_to_Index_Patient']

### Set Dataset Parameters

In [ ]:
train_eval_test_split = [0.7, 0.15, 0.15]

### Set Dataloader Parameters

In [ ]:
batch_size = 32

### Set Model Parameters

In [ ]:
num_layers = 3
num_features_in = len(features)
num_features_hidden = [64, 32, 16]
bias = True
activation = ReLU
activation_kwargs = None
dropout_p = 0.5
dropout_inplace = False
dropout_first = True
batch_norm = True
batch_norm_momentum = 0.1

### Set Trainer Parameters

In [ ]:
num_epochs = 64
learning_rate = 0.001

## Setup Environment

### Set Torch Device

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = "mps"

## Load Data

### Load Processed Training Data

In [ ]:
train_data = PopulationData().load_processed('../../data/processed/train.csv')

### Load Processed Inference Data

In [ ]:
inference_data = PopulationData().load_processed('../../data/processed/test.csv')

### Create Train Eval Test Datasets

In [ ]:
train_features, train_labels = train_data.get_feature_label_tensors(features=features)
train_dataset = DatasetMultiLayerPerceptron(features=train_features, targets=train_labels)
train_subset, eval_subset, test_subset = stratified_random_split(dataset=train_dataset,
                                                                 ratios=train_eval_test_split)

### Create Train Eval Test DataLoaders

In [ ]:
dataloader_train = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
dataloader_eval = DataLoader(eval_subset, batch_size=batch_size, shuffle=False)
dataloader_test = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

### Create Inference Dataset

In [ ]:
tensor_inference_features = inference_data.get_feature_tensor(features=features)
dataset_inference = TensorDataset(tensor_inference_features)

### Create Inference DataLoader

In [ ]:
dataloader_inference = DataLoader(dataset_inference, batch_size=batch_size, shuffle=False)

## Create Model

In [ ]:
model = ModelMultiLayerPerceptron(num_layers=num_layers,
                                  num_features_in=num_features_in,
                                  num_features_hidden=num_features_hidden,
                                  bias=bias,
                                  activation=activation,
                                  activation_kwargs=activation_kwargs,
                                  dropout_p=dropout_p,
                                  dropout_inplace=dropout_inplace,
                                  dropout_first=dropout_first,
                                  batch_norm=batch_norm,
                                  batch_norm_momentum=batch_norm_momentum,
                                  device=device)

## Create Trainer

### Create Criterion and Optimizer

In [ ]:
criterion = BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### Create Trainer

In [ ]:
trainer = TrainerMultiLayerPerceptron(model=model,
                                      criterion=criterion,
                                      optimizer=optimizer,
                                      train_loader=dataloader_train,
                                      eval_loader=dataloader_eval,
                                      device=device)

## Create Tester

## Create Runner

In [ ]:
trainer.test(test_dataset, batch_size=32, device=device)

In [ ]:
runner = Runner(model)
test_predicted = runner.predict(test_dataset, batch_size=32, device=device).to(device)
test_probabilities = runner.predict_proba(test_dataset, batch_size=32, device=device).to(device)
accuracy, precision, recall, f1, auc_roc, confusion_matrix = evaluate_tensor(test_predicted,
                                                                             test_probabilities,
                                                                             test_labels,
                                                                             device=device)
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1: {f1:.4f}')
print(f'AUC-ROC: {auc_roc:.4f}')

In [ ]:
plot_confusion_matrix(confusion_matrix, ['Healthy', 'Infected'])

In [ ]:
plot_roc(test_probabilities.cpu().numpy(), test_labels.cpu().numpy())

In [ ]:
model.save("../../models/mlp_model.pth")